In [152]:
import pandas as pd
import time
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException


In [156]:
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

In [157]:
def getInfo(link):
    driver.get(link)
    info, brand, end_date, limit, min, customer = [], [], [], [], [], []
    i = 1
    labels = WebDriverWait(driver, 10).until(
                    EC.presence_of_all_elements_located((By.XPATH, '//*[@id="list_coupon"]/div/div[5]/label'))
                )
    for label in labels: 
        label.click()
    
    screen_height = driver.execute_script("return window.screen.height;")  

    while True:
        driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))  
        i += 1
        time.sleep(5)
        scroll_height = driver.execute_script("return document.body.scrollHeight;")  
        if (screen_height) * i > scroll_height:
            break 
            
    brand1 = driver.find_elements(By.XPATH, '//*[@id="list_coupon"]/div/div[2]/a')
    brand = [el.find_element(By.TAG_NAME, 'img').get_attribute("alt") for el in brand1] + brand  


    info1 =  driver.find_elements(By.XPATH, '//*[@id="list_coupon"]/div/div[3]/p')
    info =  [s.text for s in info1] + info

    end_date1 =  driver.find_elements(By.XPATH, '//*[@id="list_coupon"]/div/div[5]/div/div[1]/div[2]')
    end_date =  [s.text for s in end_date1] + end_date

    limit1 =  driver.find_elements(By.XPATH, '//*[@id="list_coupon"]/div/div[5]/div/div[2]/div[2]')
    limit =  [s.text for s in limit1] + limit

    min1 =  driver.find_elements(By.XPATH, '//*[@id="list_coupon"]/div/div[5]/div/div[3]/div[2]')
    min =  [s.text for s in min1] + min

    customer1 =  driver.find_elements(By.XPATH, '//*[@id="list_coupon"]/div/div[5]/div/div[4]/div[2]')
    customer =  [s.text for s in customer1] + customer
    
    voucher_data = pd.DataFrame(list(zip(brand, info, end_date, limit, min, customer)),
                        columns=['brand', 'info', 'end_date', 'limit', 'min', 'customer']) 
    print("Crawl Complete")
    return voucher_data

url = 'https://iprice.vn/coupons/'
df = getInfo(url)
driver.close()

Crawl Complete


In [158]:
# Lấy file CSV
df_unique = df.drop_duplicates(keep='first')
df_unique.to_csv('D://Exam/Data.csv', index=False)
